In [ ]:
%load_ext autoreload
%autoreload 2

import os,sys
import h5py  
import numpy as np
np.seterr(invalid=['ignore','warn'][0])
import math,cmath,pickle
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.figsize'] = [6.4*1.2,4.8*1.2]
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['lines.marker'] = 's'
mpl.rcParams['lines.linestyle'] = ''
mpl.rcParams['lines.markersize'] = 12
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['xtick.major.size'] = mpl.rcParams['ytick.major.size'] = 10
mpl.rcParams['xtick.top']=mpl.rcParams['ytick.right']=True
mpl.rcParams['xtick.direction']=mpl.rcParams['ytick.direction']='in'
mpl.rcParams['legend.fontsize'] = 24
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"

# mpl.rcParams.update({"axes.grid" : True})
import util_new as yu

yu.flag_fast=True
yu.slName='plots_notes_NST'


ens='48' # 24, 24T, 48, 48T

In [ ]:
ensemble={
    '24':'cA211.530.24',
    '24T':'cA211.530.24',
    '48':'cA2.09.48',
    '48T':'cA2.09.48',
}[ens]
basepath='/p/project/pines/li47/code/projectData/NST_d-tensor/data/'
file={
    '24':'NST_a_cA211.530.24.h5',
    '24T':'NST_d_cA211.530.24_tensor.h5',
    '48':'NST_c_cA2.09.48.h5',
    '48T':'NST_d_cA2.09.48_tensor.h5',
}[ens]
data=yu.load(basepath+file)

print('Ncfg='+str(yu.deepKey(data['2pt'],2).shape[0]))
print()

Ncfg=yu.deepKey(data['2pt'],2).shape[0]
Tpack={
    '24':16,
    '24T':16,
    '48':24,
    '48T':24
}[ens]

In [ ]:
hbarc = 1/197.3

if ensemble in ['cA211.530.24']:
    pass
elif ensemble in ['cA2.09.48']:
    lat_a=0.0938; lat_N=48
    lat_amN=0.44283639
else:
    1/0

lat_aInv=1/(lat_a*hbarc)
lat_ap1=(2*math.pi)/(lat_N)

# self.tpiL=(2*math.pi)/(self.L*self.hbarc); 
#         self.mpiL=self.ampi * self.aInv * self.L * self.hbarc
#         self.mpi=self.ampi*self.aInv; self.mN=self.amN*self.aInv

In [ ]:
flags={
    'cc2pt':True, # average quantities related by complex conjugation for 2pt
    'cc3pt':True, # same for 3pt (Removal of vacuum expectation value requires 'cc2pt'=='cc3pt')
    'll2pt':True, # average irrep rows 'l1' and conjugated 'l2' (Parity breaking effect of tmQCD has been taken care of)
    'll3pt':True, # same for 3pt (This flag has no effect if 'll2pt'=True and spin-projection is done)
    'r2pt': True, # making 2ptMat real for the rest frame # 'll2pt' has to be real for this flag
    'remove_pi0VEV':True, 
}

def get2pt(opa,opb,diags=yu.diags_all):
    if opa.split(';')[-1]=='12':
        opa1=';'.join(opa.split(';')[:-1]+['n,pi+']); opa2=';'.join(opa.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get2pt(opa1,opb,diags)-np.sqrt(1/3)*get2pt(opa2,opb,diags)
    if opb.split(';')[-1]=='12':
        opb1=';'.join(opb.split(';')[:-1]+['n,pi+']); opb2=';'.join(opb.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get2pt(opa,opb1,diags)-np.sqrt(1/3)*get2pt(opa,opb2,diags)
    res=np.zeros([Ncfg,Tpack],dtype=complex)
    if opa+'_'+opb in data['2pt']:
        t=data['2pt'][opa+'_'+opb]
        res+=np.sum([t[diag] for diag in t.keys() if diag in diags],axis=0)
    if opb+'_'+opa in data['2pt']:
        t=data['2pt'][opb+'_'+opa]; 
        diags_cc={'T', 'T-pi0f'}; 
        res+=np.conj(np.sum([t[diag] for diag in t.keys() if diag in diags_cc and diag in diags],axis=0))
    return res
def get2ptMat(ops,diags=yu.diags_all):
    if flags['ll2pt']:
        flags['ll2pt']=False
        ops_flip=[yu.op_flipl(op) for op in ops]
        t=(get2ptMat(ops,diags=diags)+np.conj(get2ptMat(ops_flip,diags=diags)))/2
        flags['ll2pt']=True
        if ops[0].split(';')[1]=='0,0,0' and flags['r2pt']:
            t=np.real(t)
        return t
    t=np.transpose([[get2pt(opa,opb,diags) for opb in ops] for opa in ops],[2,3,0,1])
    if flags['cc2pt']:
        t=(t+np.conj(np.transpose(t,[0,1,3,2])))/2
    return t

def get3pt(opa,opb,insert,diags=yu.diags_all):
    if opa.split(';')[-1]=='12':
        opa1=';'.join(opa.split(';')[:-1]+['n,pi+']); opa2=';'.join(opa.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get3pt(opa1,opb,insert,diags)-np.sqrt(1/3)*get3pt(opa2,opb,insert,diags)
    if opb.split(';')[-1]=='12':
        opb1=';'.join(opb.split(';')[:-1]+['n,pi+']); opb2=';'.join(opb.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get3pt(opa,opb1,insert,diags)-np.sqrt(1/3)*get3pt(opa,opb2,insert,diags)
    res=np.zeros([Ncfg,int(insert.split('_')[-1])+1],dtype=complex)
    if opa+'_'+opb in data['3pt']:
        t=data['3pt'][opa+'_'+opb][insert]
        res+=np.sum([t[diag] for diag in t.keys() if diag in diags],axis=0)
    if opb+'_'+opa in data['3pt']:
        t=data['3pt'][opb+'_'+opa][insert]; 
        diags_cc={'B3pt','W3pt','Z3pt','T-j', 'T-pi0f-j','T-jPf','B3pt-pi0f','W3pt-pi0f','Z3pt-pi0f'}
        if opa+'_'+opb not in data['3pt'] or 'NJN-pi0f' not in data['3pt'][opa+'_'+opb][insert]:
            diags_cc.add('NJN-pi0i')
        t_add=np.zeros([Ncfg,int(insert.split('_')[-1])+1],dtype=complex)+np.sum([t[diag] for diag in t.keys() if diag in diags_cc and diag in diags],axis=0)
        res+=np.conj(t_add[:,::-1])*(yu.gtCj[insert.split('_')[0]])
    return res
def get3ptMat(opas,opbs,insert,diags=yu.diags_all):
    if flags['ll3pt']:
        flags['ll3pt']=False
        opas_flip=[yu.op_flipl(op) for op in opas]; opbs_flip=[yu.op_flipl(op) for op in opbs]
        sgns=np.array([[yu.op_getl_sgn(opa)*yu.op_getl_sgn(opb) for opb in opbs] for opa in opas])
        sgns*=yu.fourCPTstar[insert.split('_')[0]]
        t=(get3ptMat(opas,opbs,insert,diags=diags)+np.conj(get3ptMat(opas_flip,opbs_flip,insert,diags=diags))*sgns[None,None,:,:])/2
        flags['ll3pt']=True
        return t
    t=np.transpose([[get3pt(opa,opb,insert,diags) for opb in opbs] for opa in opas],[2,3,0,1])
    if flags['cc3pt']:
        flags['cc3pt']=False
        tt=get3ptMat(opbs,opas,insert,diags)[:,::-1]*(yu.gtCj[insert.split('_')[0]])
        flags['cc3pt']=True
        t=(t+np.conj(np.transpose(tt,[0,1,3,2])))/2
    return t

def remove_pi0VEV(Ctij,pi0VEV,opas,opbs):
    if not flags['remove_pi0VEV']:
        return Ctij
    
    factor=yu.coeff['g;0,0,1;G1;N1pi0,a;l1;N,pi'][0][1]
    
    ka=None # index for N(n)pi0(0)
    for i,op in enumerate(opas):
        if yu.getNpar(op)==1:
            qa=i # index for N(n)
            continue
        g,pt,irrep,occ,l,fla=op.split(';')
        if 'pi0' in occ:
            ka=i
            factor_a = factor * {'n,pi+':0,'p,pi0':1,'12':-np.sqrt(1/3)}[fla] * {'l1':1,'l2':-1}[l] * pi0VEV
    kb=None
    for i,op in enumerate(opbs):
        if yu.getNpar(op)==1:
            qb=i
            continue
        g,pt,irrep,occ,l,fla=op.split(';')
        if 'pi0' in occ:
            kb=i
            factor_b = factor * {'n,pi+':0,'p,pi0':1,'12':-np.sqrt(1/3)}[fla] * {'l1':1,'l2':-1}[l] * pi0VEV
            
    res=Ctij.copy()
    if ka is not None:
        res[:,ka,:]-=factor_a*Ctij[:,qa,:]
    if kb is not None:
        res[:,:,kb]-=np.conj(factor_b)*Ctij[:,:,qb]
    if ka is not None and kb is not None:
        res[:,ka,kb]+=factor_a*np.conj(factor_b)*Ctij[:,qa,qb]
        
    return res

pt0='0,0,0'; ofs0=[('a','p'),('N1pi1,a','12'),]; ops0=yu.getops(pt0,'l1',ofs0)
pt1='0,0,1'; ofs1=[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]; ops1=yu.getops(pt1,'l1',ofs1)

L2ops={'0':ops0,'1':ops1}
L2ptofs={'0':(pt0,ofs0),'1':(pt1,ofs1)}
L2T_pt={'0':r'$\vec{p}=(0,0,0)$','1':r'$\vec{p}=(0,0,1)$'}

L2diags={'noj':yu.diags_jLoopless,'all':yu.diags_all}
L2T_diags={'noj':'no j-loop','all':'all diags'}
diagsLs=['noj','all']

In [ ]:
# GEVP

xUnit=lat_a
fig, axs = yu.getFigAxs(4,2,sharex='col',sharey='row', gridspec_kw={'width_ratios': [2, 1]})

if ensemble in ['cA211.530.24']:
    axs[0,0].set_ylim([0,0.6]); axs[1,0].set_ylim([0,0.5])
    axs[2,0].set_ylim([0,0.1]); axs[3,0].set_ylim([0,0.1])
elif ensemble in ['cA2.09.48']:
    axs[0,0].set_ylim([0,0.06]); axs[1,0].set_ylim([0,0.15])
    axs[2,0].set_ylim([0,0.02]); axs[3,0].set_ylim([0,0.03])

dtt0=4
fitminmin=7
chi2Size=12

vev_pi0f_jk=yu.jackknife(data['VEV']['pi0f'])

def jack_remove_pi0VEV(Ctij,opas,opbs):
    return np.array([remove_pi0VEV(cMat,vev,opas,opbs) for cMat,vev in zip(Ctij,vev_pi0f_jk)])

opsL='0'
pt,ofs=L2ptofs[opsL]
Nop=len(ofs)
ops=yu.getops(pt,'l1',ofs)
c2ptMat_0_dat=get2ptMat(ops)
c2ptMat_0_jk=yu.jackknife(c2ptMat_0_dat)

def func(c2ptMat_jk):
    eVals_jk,eVecs_jk=yu.jackmap(lambda c2ptMat:yu.GEVP(c2ptMat,-dtt0), c2ptMat_jk)
    eVecs_jk=yu.jackmap(yu.renormalize_eVecs, eVecs_jk)
    eVec_jk=eVecs_jk[:,:,0,:].T
    eVec_jk=eVec_jk/eVec_jk[0]
    eVec_jk=np.transpose(eVec_jk,[2,0,1])
    eVecs_inv_jk=np.linalg.inv(eVecs_jk)
    W_jk=1/(eVecs_jk[:,:,0,0]*eVecs_inv_jk[:,:,0,0])-1
    return eVec_jk,W_jk
eVec_0_jk,W_0_jk=func(c2ptMat_0_jk)

eVec_0_mean,eVec_0_err=yu.jackme(np.abs(eVec_0_jk))
W_0_mean,W_0_err=yu.jackme(np.abs(W_0_jk))

irow=0; icol=0
tmin=dtt0+1; tmax=len(eVec_0_mean[0])
label=r'$i=N(1)\pi(1)$'
plt_x=np.arange(tmin,tmax)*xUnit; plt_y=eVec_0_mean[1,tmin:tmax]; plt_err=eVec_0_err[1,tmin:tmax]
axs[irow,icol].errorbar(plt_x,plt_y,plt_err,color=['r','g','b'][1],fmt=['s','d','o'][1],mfc='white',label=label)

irow=1; icol=0
tmin=dtt0+1; tmax=len(eVec_0_mean[0])
plt_x=np.arange(tmin,tmax)*xUnit; plt_y=W_0_mean[tmin:tmax]; plt_err=W_0_err[tmin:tmax]
axs[irow,icol].errorbar(plt_x,plt_y,plt_err,color=['r','g','b'][0],fmt=['s','d','o'][0],mfc='white',label=label)

relas=np.abs(eVec_0_err/eVec_0_mean)
temp=[(i,rela) for i,rela in enumerate(relas.T) if np.any(rela>0.2)]
fitmax=temp[0][0]
relas=np.abs(W_0_err/W_0_mean)
temp=[(i,rela) for i,rela in enumerate(relas) if rela>0.2]
fitmax=min(fitmax,temp[0][0])

fits=[]
fitmins=np.arange(fitminmin,fitmax-1)

dic=[]

pars0=[1,1]; Npar=len(pars0)
for fitmin in fitmins:
    fitsizeunit=fitmax-fitmin
    def fitfunc(pars):
        Wr,v1r=pars
        return [Wr]*fitsizeunit + [v1r]*fitsizeunit
    y_jk=np.concatenate([np.real(W_0_jk[:,fitmin:fitmax]),np.real(eVec_0_jk[:,1,fitmin:fitmax])],axis=1)
    obs_jk,chi2R,Ndof=yu.jackfit(fitfunc,y_jk,pars0)
    dic.append(obs_jk)
    (obs_mean,obs_err)=yu.jackme(obs_jk)
    pars0=obs_mean[-Npar:]
    fits.append([obs_mean,obs_err,chi2R,Ndof])
    
    irow=0; icol=1
    plt_x=fitmin*xUnit; plt_y=obs_mean[1]; plt_err=obs_err[1]
    axs[irow,icol].errorbar(plt_x,plt_y,plt_err,color=['r','g','b'][1],fmt=['s','d','o'][1],mfc='white')
    ylim=axs[irow,icol].get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
    axs[irow,icol].annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=['r','g','b'][1],size=chi2Size,ha='center')
    
    irow=1; icol=1
    plt_x=fitmin*xUnit; plt_y=obs_mean[0]; plt_err=obs_err[0]
    axs[irow,icol].errorbar(plt_x,plt_y,plt_err,color=['r','g','b'][0],fmt=['s','d','o'][0],mfc='white')
    ylim=axs[irow,icol].get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
    axs[irow,icol].annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=['r','g','b'][0],size=chi2Size,ha='center')
    
obs_mean_MA,obs_err_MA,probs=yu.modelAvg(fits)
ind=np.argmax(probs)
WG_0_jk=dic[ind][:,0]; eVecG_0_jk=np.concatenate([np.zeros((len(dic[ind]),1))+1,dic[ind][:,1:2]],axis=1)

irow=0; icol=1
plt_x=fitmins[ind]*xUnit; plt_y=fits[ind][0][1]; plt_err=fits[ind][1][1]
axs[irow,icol].errorbar(plt_x,plt_y,plt_err,color=['r','g','b'][1],fmt=['s','d','o'][1])
irow=1; icol=1
plt_x=fitmins[ind]*xUnit; plt_y=fits[ind][0][0]; plt_err=fits[ind][1][0]
axs[irow,icol].errorbar(plt_x,plt_y,plt_err,color=['r','g','b'][0],fmt=['s','d','o'][0])


opsL='1'
pt,ofs=L2ptofs[opsL]
Nop=len(ofs)
ops=yu.getops(pt,'l1',ofs)
c2ptMat_1_dat=get2ptMat(ops)
c2ptMat_1_jk=yu.jackknife(c2ptMat_1_dat)
c2ptMat_1_jk=jack_remove_pi0VEV(c2ptMat_1_jk,ops,ops)

def func(c2ptMat_jk):
    eVals_jk,eVecs_jk=yu.jackmap(lambda c2ptMat:yu.GEVP(c2ptMat,-dtt0), c2ptMat_jk)
    eVecs_jk=yu.jackmap(yu.renormalize_eVecs, eVecs_jk)
    eVec_jk=eVecs_jk[:,:,0,:].T
    eVec_jk=eVec_jk/eVec_jk[0]
    eVec_jk=np.transpose(eVec_jk,[2,0,1])
    eVecs_inv_jk=np.linalg.inv(eVecs_jk)
    W_jk=1/(eVecs_jk[:,:,0,0]*eVecs_inv_jk[:,:,0,0])-1
    return eVec_jk,W_jk
eVec_1_jk,W_1_jk=func(c2ptMat_1_jk)

eVec_1_mean,eVec_1_err=yu.jackme(np.abs(eVec_1_jk))
W_1_mean,W_1_err=yu.jackme(np.abs(W_1_jk))

irow=2; icol=0
tmin=dtt0+1; tmax=len(eVec_1_mean[0])
label=r'$i=N(1)\pi(1)$'
for ind in [1,2]:
    plt_x=np.arange(tmin,tmax)*xUnit; plt_y=eVec_1_mean[ind,tmin:tmax]; plt_err=eVec_1_err[ind,tmin:tmax]
    axs[irow,icol].errorbar(plt_x,plt_y,plt_err,color=['r','g','b'][ind],fmt=['s','d','o'][ind],mfc='white',label=label)

irow=3; icol=0
tmin=dtt0+1; tmax=len(eVec_1_mean[0])
plt_x=np.arange(tmin,tmax)*xUnit; plt_y=W_1_mean[tmin:tmax]; plt_err=W_1_err[tmin:tmax]
axs[irow,icol].errorbar(plt_x,plt_y,plt_err,color=['r','g','b'][0],fmt=['s','d','o'][0],mfc='white',label=label)

relas=np.abs(eVec_1_err/eVec_1_mean)
temp=[(i,rela) for i,rela in enumerate(relas.T) if np.any(rela>0.2)]
fitmax=temp[0][0]
relas=np.abs(W_1_err/W_1_mean)
temp=[(i,rela) for i,rela in enumerate(relas) if rela>0.2]
fitmax=min(fitmax,temp[0][0])

fits=[]
fitmins=np.arange(fitminmin,fitmax-1)

dic=[]

pars0=[1,1,1,1,1,1]; Npar=len(pars0)
for fitmin in fitmins:
    fitsizeunit=fitmax-fitmin
    def fitfunc(pars):
        Wr,v1r,v2r,Wi,v1i,v2i=pars
        return [Wr]*fitsizeunit + [v1r]*fitsizeunit + [v2r]*fitsizeunit + [Wi]*fitsizeunit + [v1i]*fitsizeunit + [v2i]*fitsizeunit
    y_jk=np.concatenate([np.real(W_1_jk[:,fitmin:fitmax]),np.imag(W_1_jk[:,fitmin:fitmax]),\
        np.real(eVec_1_jk[:,1,fitmin:fitmax]),np.imag(eVec_1_jk[:,1,fitmin:fitmax]),\
            np.real(eVec_1_jk[:,2,fitmin:fitmax]),np.imag(eVec_1_jk[:,2,fitmin:fitmax])],axis=1)
    obs_jk,chi2R,Ndof=yu.jackfit(fitfunc,y_jk,pars0)
    (obs_mean,obs_err)=yu.jackme(obs_jk)
    pars0=obs_mean[-Npar:]
    obs_jk=np.array([obs_jk[:,0]+1j*obs_jk[:,1],obs_jk[:,2]+1j*obs_jk[:,3],obs_jk[:,4]+1j*obs_jk[:,5]]).T
    dic.append(obs_jk)
    (obs_mean,obs_err)=yu.jackme(np.abs(obs_jk))
    fits.append([obs_mean,obs_err,chi2R,Ndof])
    
    irow=2; icol=1
    for ind in [1,2]:
        plt_x=fitmin*xUnit; plt_y=obs_mean[ind]; plt_err=obs_err[ind]
        axs[irow,icol].errorbar(plt_x,plt_y,plt_err,color=['r','g','b'][ind],fmt=['s','d','o'][ind],mfc='white')
        ylim=axs[irow,icol].get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
        axs[irow,icol].annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=['r','g','b'][ind],size=chi2Size,ha='center')
    
    irow=3; icol=1
    plt_x=fitmin*xUnit; plt_y=obs_mean[0]; plt_err=obs_err[0]
    axs[irow,icol].errorbar(plt_x,plt_y,plt_err,color=['r','g','b'][0],fmt=['s','d','o'][0],mfc='white')
    ylim=axs[irow,icol].get_ylim(); chi2_shift=(ylim[1]-ylim[0])/12
    axs[irow,icol].annotate("%0.1f" %chi2R,(plt_x,plt_y-plt_err-chi2_shift),color=['r','g','b'][0],size=chi2Size,ha='center')
    
obs_mean_MA,obs_err_MA,probs=yu.modelAvg(fits)
ind=np.argmax(probs)
WG_1_jk=dic[ind][:,0]; eVecG_1_jk=np.concatenate([np.zeros((len(dic[ind]),1))+1,dic[ind][:,1:3]],axis=1)

irow=2; icol=1
for ind2 in [1,2]:
    plt_x=fitmins[ind]*xUnit; plt_y=fits[ind][0][ind2]; plt_err=fits[ind][1][ind2]
    axs[irow,icol].errorbar(plt_x,plt_y,plt_err,color=['r','g','b'][ind2],fmt=['s','d','o'][ind2])
irow=3; icol=1
plt_x=fitmins[ind]*xUnit; plt_y=fits[ind][0][0]; plt_err=fits[ind][1][0]
axs[irow,icol].errorbar(plt_x,plt_y,plt_err,color=['r','g','b'][0],fmt=['s','d','o'][0])

plt.tight_layout()
plt.savefig('fig/2ptGEVP_'+ens+'.pdf')
plt.close()

In [ ]:
# c2pt and c2ptG (GEVP)

xUnit=lat_a; yUnit=lat_aInv
fig, axd = plt.subplot_mosaic([['f1','f1','f1'],['f2','f2','f3']],figsize=(20,8))
(ax1,ax2,ax3)=(axd[key] for key in ['f1','f2','f3'])

c2pt_0_jk=np.real(c2ptMat_0_jk[:,:,0,0])
c2ptG_0_jk=np.real(np.einsum('ci,ctij,cj->ct',np.conj(eVecG_0_jk),c2ptMat_0_jk,eVecG_0_jk))

mEff_0_jk=np.log(c2pt_0_jk/np.roll(c2pt_0_jk,-1,axis=1))
mean,err=yu.jackme(mEff_0_jk)
tmin=1; tmax=len(mean)-1
plt_x=np.arange(tmin,tmax)*xUnit; plt_y=mean[tmin:tmax]; plt_err=err[tmin:tmax]
ax1.errorbar(plt_x,plt_y,plt_err,color='b',fmt='s',mfc='white')

mEffG_0_jk=np.log(c2ptG_0_jk/np.roll(c2ptG_0_jk,-1,axis=1))
mean,err=yu.jackme(mEffG_0_jk)
plt_x=np.arange(tmin,tmax)*xUnit; plt_y=mean[tmin:tmax]; plt_err=err[tmin:tmax]
ax1.errorbar(plt_x,plt_y,plt_err,color='r',fmt='s',mfc=None)


plt.tight_layout()
plt.savefig('fig/c2pt_0_'+ens+'.pdf')
plt.close()

# mom 1

xUnit=lat_a; yUnit=lat_aInv
fig, axd = plt.subplot_mosaic([['f1','f1','f1'],['f2','f2','f3']],figsize=(20,8))
(ax1,ax2,ax3)=(axd[key] for key in ['f1','f2','f3'])

c2pt_1_jk=np.real(c2ptMat_1_jk[:,:,0,0])
c2ptG_1_jk=np.real(np.einsum('ci,ctij,cj->ct',np.conj(eVecG_1_jk),c2ptMat_1_jk,eVecG_1_jk))

mEff_1_jk=np.log(c2pt_1_jk/np.roll(c2pt_1_jk,-1,axis=1))
mean,err=yu.jackme(mEff_1_jk)
tmin=1; tmax=len(mean)-1
plt_x=np.arange(tmin,tmax)*xUnit; plt_y=mean[tmin:tmax]; plt_err=err[tmin:tmax]
ax1.errorbar(plt_x,plt_y,plt_err,color='b',fmt='s',mfc='white')

mEffG_1_jk=np.log(c2ptG_1_jk/np.roll(c2ptG_1_jk,-1,axis=1))
mean,err=yu.jackme(mEffG_1_jk)
plt_x=np.arange(tmin,tmax)*xUnit; plt_y=mean[tmin:tmax]; plt_err=err[tmin:tmax]
ax1.errorbar(plt_x,plt_y,plt_err,color='r',fmt='s',mfc=None)


plt.tight_layout()
plt.savefig('fig/c2pt_1_'+ens+'.pdf')
plt.close()


In [ ]:
# FF: (pips,)
FF2info={
    'RS_0_00':(
        [(1,'id','P0')]
        ),
}

def get_dt(eVeca_li,Wa_l,eVecb_li,Wb_l,Itype='Id'):
    dt2pta_lij=np.einsum('li,lj->lij',np.conj(eVeca_li),eVeca_li)
    if flags['dt2pt']:
        dt2pta_lij[:,1:,1:]*=0
        dt2pta_lij[:,0,0]*=1-np.conj(Wa_l)*Wa_l
        dt2pta_lij[:,1:,0]*=1+Wa_l[:,None]
        dt2pta_lij[:,0,1:]*=1+np.conj(Wa_l[:,None])

    dt2ptb_lij=np.einsum('li,lj->lij',np.conj(eVecb_li),eVecb_li)
    if flags['dt2pt']:
        dt2ptb_lij[:,1:,1:]*=0
        dt2ptb_lij[:,0,0]*=1-np.conj(Wb_l)*Wb_l
        dt2ptb_lij[:,1:,0]*=1+Wb_l[:,None]
        dt2ptb_lij[:,0,1:]*=1+np.conj(Wb_l[:,None])

    dt3pt_ijlm=np.einsum('li,mj->ijlm',np.conj(eVeca_li),eVecb_li)
    if Itype in ['Id','Ip']:
        dt3pt_ijlm[1:,1:]*=0
    if flags['dt3pt'] and Itype in ['Id']:
        d00_lm=1-np.einsum('l,m->lm',np.conj(Wa_l),Wb_l)
        di0_lm=1+np.einsum('l,m->lm',[1,1],Wb_l)
        d0j_lm=1+np.einsum('l,m->lm',np.conj(Wa_l),[1,1])
        dt3pt_ijlm[1:,1:]*=0
        dt3pt_ijlm[0,0]*=d00_lm
        dt3pt_ijlm[1:,0]*=di0_lm[None,:,:]
        dt3pt_ijlm[0,1:]*=d0j_lm[None,:,:]

    return (dt2pta_lij,dt2ptb_lij,dt3pt_ijlm)

In [ ]:
tfs=[10,12,14]; diags=yu.diags_all
def run(FFs):
    xUnit=lat_a
    fig, axs = yu.getFigAxs(len(FFs),2,sharex=True)
    
    for FF in FFs:
        pips=FF2info[FF]
        for j in ['j+','j-']:        
            needsVEV = (FF,j) in [('RS_0_00','j+')]   
            
            for case in ['noGEVP','GEVP']:
                opasL,opbsL=FF.split('_')[-1]
                ptofas=L2ptofs[opasL]; ptofbs=L2ptofs[opbsL]
                pta,ofas=ptofas; ptb,ofbs=ptofbs
                if case=='noGEVP':
                    ofas=[ofas[0]]; ofbs=[ofbs[0]]
                def t_get(tf,t_insert):
                    t=[[jack_remove_pi0VEV(
                        get3ptMat(yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs),t_insert+'_'+str(tf),diags)[:,1:tf],
                        yu.getops(pta,la,ofas),yu.getops(pta,lb,ofas))\
                        for lb in ['l1','l2']] for la in ['l1','l2']]
                    return yu.jackknife(np.transpose(t,[2,3,4,5,0,1])) # cfg,tc,opa,opb,la,lb
                
                c3pt_jk_list=[t_get(tf,insert+'_'+j) for prefactor,insert,proj in pips for tf in tfs]
                
                if case=='noGEVP':
                    dt2pta_lij,dt2ptb_lij,dt3pt_ijlm=(np.zeros((2,1,1))+1,np.zeros((2,1,1))+1,np.zeros((1,1,2,2))+1)
                else:
                    
                    dt2pta_lij,dt2ptb_lij,dt3pt_ijlm=get_dt(eVeca_li,Wa_l,eVecb_li,Wb_l)
                
                if needsVEV:
                    proj=proj2mat[pips[0][2]]
                    t2pt_sub=np.einsum('tij,ij->t',t2pta_ltij[0],dt3pt_ijlm[:,:,0,0])*proj[0,0]+\
                        np.einsum('tij,ij->t',t2pta_ltij[1],dt3pt_ijlm[:,:,1,1])*proj[1,1]
                
                AInv=yu.getCoeMat(pta); BInv=yu.getCoeMat(ptb)
            
run(['RS_0_00'])